In [15]:
import requests
import json
import pandas as pd

In [16]:
ratings = pd.read_csv('data/articles_ratings.csv', index_col=0)
ratings.head()

,Rating
Article,
Datenschutzrecht,4.23
Richtlinie 95/46/EG (Datenschutzrichtlinie),4.08
Datenschutz,4.08
Datenschutzerklärung,4.00
Datenschutz-Grundverordnung,3.92


In [17]:
all_data = []
for article in ratings.index:
    rating = ratings.loc[article, 'Rating']
    url = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/user/{}/monthly/20141001/20171031'.format(article)
    r = requests.get(url)
    if r.status_code == 200:
        data = r.json()
        article_data = data['items']
        for item in article_data:
            item['rating'] = rating
        all_data += article_data
    else:
        print('Problem with article "{}".'.format(article))


with open('data/pageviews.json', 'w') as f:
    json.dump(all_data, f)

Problem with article "Richtlinie 95/46/EG (Datenschutzrichtlinie)".
Problem with article "Google LLC".
Problem with article "Agentur der Europäischen Union für Cybersicherheit".
Problem with article "GAFAM".
Problem with article "Brüssel-Effekt".
Problem with article "Meme (Kulturphänomen)".
Problem with article "Verordnung (EU) Nr. 910/2014 (eIDAS-Verordnung)".
Problem with article "Royal Assent".
Problem with article "Tronc (Unternehmen)".
Problem with article "NOYB".


In [3]:
all_data = []
with open('data/articles_3.0.txt', 'r') as f:
    lines = f.readlines()
    for article in lines:
        url = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/user/{}/monthly/20141001/20171031'.format(article[:-1])
        r = requests.get(url)
        if r.status_code == 200:
            data = r.json()
            all_data += data['items']
        else:
            print("Problem with article {}.".format(article[:-1]))


with open('data/pageviews_3.0.json', 'w') as f:
    json.dump(all_data, f)

Problem with article Richtlinie 95/46/EG (Datenschutzrichtlinie).
Problem with article Google LLC.


In [4]:
article_data = data['items']
for item in article_data:
    item['rating'] = 1
article_data

[{'project': 'de.wikipedia',
  'article': 'Verordnung_(EU)',
  'granularity': 'monthly',
  'timestamp': '2015070100',
  'access': 'all-access',
  'agent': 'user',
  'views': 4574,
  'rating': 1},
 {'project': 'de.wikipedia',
  'article': 'Verordnung_(EU)',
  'granularity': 'monthly',
  'timestamp': '2015080100',
  'access': 'all-access',
  'agent': 'user',
  'views': 3797,
  'rating': 1},
 {'project': 'de.wikipedia',
  'article': 'Verordnung_(EU)',
  'granularity': 'monthly',
  'timestamp': '2015090100',
  'access': 'all-access',
  'agent': 'user',
  'views': 5069,
  'rating': 1},
 {'project': 'de.wikipedia',
  'article': 'Verordnung_(EU)',
  'granularity': 'monthly',
  'timestamp': '2015100100',
  'access': 'all-access',
  'agent': 'user',
  'views': 5508,
  'rating': 1},
 {'project': 'de.wikipedia',
  'article': 'Verordnung_(EU)',
  'granularity': 'monthly',
  'timestamp': '2015110100',
  'access': 'all-access',
  'agent': 'user',
  'views': 5828,
  'rating': 1},
 {'project': 'de.wik

In [5]:
all_data

[{'project': 'de.wikipedia',
  'article': 'Datenschutzrecht',
  'granularity': 'monthly',
  'timestamp': '2015070100',
  'access': 'all-access',
  'agent': 'user',
  'views': 1078},
 {'project': 'de.wikipedia',
  'article': 'Datenschutzrecht',
  'granularity': 'monthly',
  'timestamp': '2015080100',
  'access': 'all-access',
  'agent': 'user',
  'views': 951},
 {'project': 'de.wikipedia',
  'article': 'Datenschutzrecht',
  'granularity': 'monthly',
  'timestamp': '2015090100',
  'access': 'all-access',
  'agent': 'user',
  'views': 1218},
 {'project': 'de.wikipedia',
  'article': 'Datenschutzrecht',
  'granularity': 'monthly',
  'timestamp': '2015100100',
  'access': 'all-access',
  'agent': 'user',
  'views': 1541},
 {'project': 'de.wikipedia',
  'article': 'Datenschutzrecht',
  'granularity': 'monthly',
  'timestamp': '2015110100',
  'access': 'all-access',
  'agent': 'user',
  'views': 1670},
 {'project': 'de.wikipedia',
  'article': 'Datenschutzrecht',
  'granularity': 'monthly',
 

In [4]:
# To read data
df = pd.read_json('data/pageviews_2.5.json')
df

,project,article,granularity,timestamp,access,agent,views
0,de.wikipedia,Datenschutzrecht,monthly,2033-11-08 13:41:40,all-access,user,1078
1,de.wikipedia,Datenschutzrecht,monthly,2033-11-08 16:28:20,all-access,user,951
2,de.wikipedia,Datenschutzrecht,monthly,2033-11-08 19:15:00,all-access,user,1218
3,de.wikipedia,Datenschutzrecht,monthly,2033-11-08 22:01:40,all-access,user,1541
4,de.wikipedia,Datenschutzrecht,monthly,2033-11-09 00:48:20,all-access,user,1670
...,...,...,...,...,...,...,...
1155,de.wikipedia,Verordnung_(EU),monthly,2033-12-01 14:28:20,all-access,user,4894
1156,de.wikipedia,Verordnung_(EU),monthly,2033-12-01 17:15:00,all-access,user,4784
1157,de.wikipedia,Verordnung_(EU),monthly,2033-12-01 20:01:40,all-access,user,4311
1158,de.wikipedia,Verordnung_(EU),monthly,2033-12-01 22:48:20,all-access,user,4977
